In [2]:
import math
from pandas import DataFrame
from collections import Counter
import matplotlib.pyplot as plt

In [3]:
def ganador(tabla:DataFrame) -> str :
    '''
    Funcion que indica el equipo ganador en una serie de partidos
    Entrada: 
            tabla: Dataframe cuyas columnas con indice 5 y 6 son el numero de
            goles que metido el equipo local y el visitantes respectivamente
    
    Salida: El papel que juega el equipo que ganó, es decir, visitante o local.
            En caso de que haya un empate se regresa "Empate"
    '''
    if tabla[5] < tabla[6]:
        return "Visitante"
    elif tabla[6] < tabla[5]:
        return "Local"
    elif tabla[6] == tabla[5]:
        return "Empate"

In [4]:

def poisson (lamd:float, goles:int, )->float:
    '''
    Funcion de distribucion Poisson
    Entrada:
            lambd: Parametro lambda en la funcion de distribucion Poisson
            goles: Valor que va a tomar la variable aleatoria 

    Salida:
            resultado: Probabilidad de que la variable aleatoria tome el valor "goles"
    '''
    resultado = (((math.exp(-lamd)) * (lamd**goles)) / math.factorial(goles))
    return resultado


In [5]:
def prediccion(final:DataFrame) -> str:
    '''
    Funcion que decide el resultado de un partido
    Entrada:
            final: Dataframe que contiene las probabilidades de que gane el
            equipo local, el visitante o de que haya empate

    Salida:
            La prediccion del resultado del partido
    '''
    #creamso la columna 'prediccion'
    final['prediccion'] = 'Nan'
    
    #buscamos los lugares donde la probabilidad del local es mayor que
    # la del visitante y le asignamos como resultado local
    mascara_local = final['proba_local'] > final['proba_visitante']
    final.loc[mascara_local, 'prediccion'] = 'Local'

    #hacemos lo contrario que arriba
    mascara_visitante = final['proba_local'] < final['proba_visitante']
    final.loc[mascara_visitante, 'prediccion'] = 'Visitante'

    #definimos la condicion para que haya empate y la asignamos
    mascara_empate = ((final['proba_empate'] > final['proba_visitante']) & (final['proba_empate'] > final['proba_local'])) |(abs(final['proba_local'] - final['proba_visitante']) < 0.2)
    final.loc[mascara_empate, 'prediccion'] = 'Empate'

    return final
    


In [6]:
def loc_vis(temp_entr:str, historico:DataFrame):

    '''
    Funcion que dada una temparada regresa dos tablas que contienen
    el promedio de goles a favor y encontra del equipo local y visitantez

    Entrada:
            temp_entr: Temporada con la cual se van a hacer los calculos
            historico: Dataframe que contiene todos los partidos

    Salida:
            visitantes_promedio: dataframe con el promedio de goles a favor
            y en contra de los equipos cuando son locales, ademas de sus
            facotores de ataque y defensa

            locales_promedio: Dataframe igual que visitantes_promedio
            pero con los datos de los equipos como locales
    '''

    #tomamos todos los partidos de la temporada temp_entr y de la division 1
    partidos_entr = historico[(historico['temporada'] == temp_entr) & (historico['division'] == 1)]

    #copiamos la tabla para locales y para visitantes
    locales = partidos_entr.copy()
    visitantes = partidos_entr.copy()

    #eliminamos la columna 'visitante' de la tabla 'locales'
    # y hacemos lo analogo con la nueva tabla 'visitantes'
    locales = locales.drop('visitante', 1)
    visitantes = visitantes.drop('local', 1)

    #renombramos las columnas para que tengan sentido
    visitantes = visitantes.rename(columns = {
        'visitante' : 'equipo',
        'localGoles' : 'goles_contra',
        'vistanteGoles' : 'goles_favor'
    })
    locales = locales.rename(columns = {
    'local' : 'equipo',
    'localGoles' : 'goles_favor',
    'vistanteGoles' : 'goles_contra'
    })

    #agrupamos los partidos por equipo y lo hacemos en cada tabla
    locales_promedio = locales.groupby(['equipo'])
    visitantes_promedio = visitantes.groupby(['equipo'])

    #obtenemos los promedios de goles a favor y en contra de cada tabla
    locales_promedio = locales_promedio.agg({
        'goles_favor' : 'mean',
        'goles_contra' : 'mean'
    })
    visitantes_promedio = visitantes_promedio.agg({
        'goles_favor' : 'mean',
        'goles_contra' : 'mean'
    })

    #renombramos las nuevas columnas de los promedios obtenidos
    locales_promedio = locales_promedio.rename(columns = {
        'goles_favor' : 'prom_gol_favor_local',
        'goles_contra' : 'prom_gol_cont_local'
    })

    visitantes_promedio = visitantes_promedio.rename(columns = {
        'goles_favor' : 'prom_gol_favor_vis',
        'goles_contra' : 'prom_gol_cont_vis'
    })

    #regresamos las tablas
    return visitantes_promedio, locales_promedio

In [7]:
def promedios(locales_promedio: DataFrame, visitantes_promedio: DataFrame, historico: DataFrame, temp_entr:str)-> float:
        '''
        Funcion que calcula los facotres de ataque y defensa de los equipos
        locales y visitantes
        
        Entrada:
                tabla: Dataframe que contiene las columnas vistianteGoles
                y localGoles, que indican el numero de goles que mete cada 
                uno de ellos

        Salida:
                visitantes_promedio: dataframe con los factores mencionados 
                para los visitantes
                locales_promedio: dataframe con los factores mencionados 
                para los locales
        '''
        #obtenemos el promedio de goles que meten los 
        # locales y los visitantes en la temporadad de entrenamiento 
        temp = historico[historico['temporada'] == temp_entr]
        promGolV = temp['vistanteGoles'].mean()
        promGolL = temp['localGoles'].mean()

        #calculamos los factores de ataque y defensa respecto
        locales_promedio['fac_ataqueL'] = locales_promedio['prom_gol_favor_local']/promGolL
        locales_promedio['fac_defensaL'] = locales_promedio['prom_gol_cont_local']/promGolV

        visitantes_promedio['fac_ataqueV'] = visitantes_promedio['prom_gol_favor_vis']/promGolL
        visitantes_promedio['fac_defensaV'] = visitantes_promedio['prom_gol_cont_vis']/promGolV

        return visitantes_promedio, locales_promedio

In [8]:
def tablas_combinadas(temp_predecir:str,locales_promedio:DataFrame, visitantes_promedio:DataFrame, historico:DataFrame) -> DataFrame:
        '''
        Funcion que cruza las tamblas temp_predecir y locales_promedio
        y se queda con los datos base y los factores de ataque y defensa
        de los locales y visitantes

        Entrada:
                temp_predecir: Temporada a predecir
                
                locales_promedio: Dataframe que contiene los factores de 
                ataque y defensa de los equipos como locales
                
                visitantes_promedio: Dataframe que contiene los factores de 
                ataque y defensa de los equipos como visitantes
                
                historico: Dataframe con todos los partidos de las temporadas

        Salida:
                combinar2: Dataframe con los datos base y los facotes de 
                atque y defensa de cada equipo en su papel de visitante
                y de local
        '''


        #la tabla 'predecir' va a contener la tempotara de partidos a pronosticar
        predecir = historico.copy()
        predecir = predecir[(predecir['temporada'] == temp_predecir) & (predecir['division'] == 1) ]
        
        #juntamos las tablas dadas por medio de la columna en comun llamada 'equipo' y la llamamos 'total'
        total = locales_promedio.merge(visitantes_promedio, on = ['equipo'])
        
        #creamos la tabla combinar1 combinando la tabla 'predecir' y total 
        combinar1 = predecir.merge(total, left_on = 'local', right_on = 'equipo')
        #solo nos quedamos con los datos base y los factores de ataque y defensa del local 
        # de los partidos a predecir


        combinar1 = combinar1.drop(['division',
                        'prom_gol_favor_local', 'prom_gol_cont_local',
                        'prom_gol_favor_vis', 'prom_gol_cont_vis',
                        'fac_ataqueV', 'fac_defensaV'], 1)

        #combinamos la tabla 'combinar1' con la tabla total por medio de 'visitante' = 'equipo' y
        # nos quedamos con los datos que ya teniamos en combinar1 mas los del factor de ataque y 
        # defensa del visitante
        combinar2 = combinar1.merge(total, left_on = 'visitante', right_on = 'equipo')
        combinar2 = combinar2.drop(['prom_gol_favor_local', 'prom_gol_cont_local',
                        'prom_gol_favor_vis', 'prom_gol_cont_vis', 'fac_ataqueL_y', 'fac_defensaL_y'], 1)
        
        return combinar2


In [9]:
def tab_lamb(combinar2:DataFrame, historico:DataFrame, temp_entr:str)->DataFrame:
        '''
        Funcion que calcula el valor lambda para la distribucion Poisson

        Entrada:
                combinar2: Dataframe con los datos base y los facotes de 
                atque y defensa de cada equipo en su papel de visitante
                y de local

                historico:  Dataframe con todos los partidos de las temporadas

                temp_entr: Temporada para entrenar al modelo

        Salida:
                pre_final: Dataframe con los valores lambda para el 
                visitante y local

                goles_max: El numero maximo de goles que se han metido
                desde la temporada 1970-71
        '''
        #copiamos la tabla combinar2
        lambdas = combinar2.copy()

        #promedios de todos los partidos de la temporada de entrenamiento
        temp = historico[historico['temporada'] == temp_entr]
        promGolV = temp['vistanteGoles'].mean()
        promGolL = temp['localGoles'].mean()
        #obtenemos los promedios ponderados
        lambdas['lambdaL'] = lambdas['fac_ataqueL_x'] + lambdas['fac_defensaV'] * promGolL
        lambdas['lambdaV'] = lambdas['fac_ataqueV'] + lambdas['fac_defensaL_x'] * promGolV

        #obtenemos el maximo de goles y el minimo de goles anotados por los locales, los
        # cuales sabemos que anotan mas
        goles_min = historico['localGoles'].min()
        goles_max = historico['localGoles'].max()

        #corremos sobre el rango de goles
        for i in range(goles_min, goles_max + 1):
                #funcion poisson para la probabilidad de que metan i goles
                lambdas[f'proba_{i}GolesL'] = lambdas['lambdaL'].apply(poisson, goles = i)
                lambdas[f'proba_{i}GolesV'] = lambdas['lambdaV'].apply(poisson, goles = i)
        
        #tiramos las columnas que ya no se usan
        pre_final = lambdas.copy()
        pre_final = pre_final.drop(['fac_ataqueL_x', 'fac_defensaL_x',
                'fac_ataqueV', 'fac_defensaV', 'lambdaL', 'lambdaV'], 1)
        
        return pre_final, goles_max


In [10]:
def resultado(temp_entr:str, temp_predecir:str, historico:DataFrame) -> DataFrame:
        '''
        Funcion pronostica que equipo va a ganar

        Entrada:
                temp_entr: Temporada para entrenar al modelo

                temp_predecir: Temporada a predecir

                historico: Dataframe con todos los partidos de las temporadas
                
        Salida:
                precision: Precision del modelo

                final: Dataframe que contiene la prediccion del modelo
        '''
        #obtenemos los promedios de goles de cada equipo, ademas de sus factores
        # de ataque y defensa
        visitantes_promedio, locales_promedio = loc_vis(temp_entr, historico)
        visitantes_promedio, locales_promedio = promedios(locales_promedio, visitantes_promedio, historico, temp_entr)

        #mandamos llamar a la funcion tabalas_combinadas
        combinar2 = tablas_combinadas(temp_predecir,locales_promedio, visitantes_promedio, historico)
        
        #obtenemos el dataframe con lo datos para poder pronosticar que equipo va a ganar
        pre_final, goles_max= tab_lamb(combinar2, historico, temp_entr)

        #Vamos a sumar todas las probabilidades donde el local gana, cuando el
        # visitante gana o cuando hay un empate
        pre_final['proba_local'] = 0
        pre_final['proba_visitante'] = 0
        pre_final['proba_empate'] = 0

        #corremos sobre la matriz y sumamos las probabilidades
        for i in range(goles_max + 1):
            for j in range(goles_max + 1):
                #sumamos los valores fuera de la diagonal
                if i != j and j > 1:
                    pre_final['proba_local'] = pre_final['proba_local'] + (pre_final[f'proba_{j}GolesL'] * pre_final[f'proba_{i}GolesV'])
                    pre_final['proba_visitante'] = pre_final['proba_visitante'] + (pre_final[f'proba_{i}GolesL'] * pre_final[f'proba_{j}GolesV'])
                #sumamos la diagonal para los empates
                elif i == j:
                    pre_final['proba_empate'] = pre_final['proba_empate'] + (pre_final[f'proba_{i}GolesL'] * pre_final[f'proba_{j}GolesV'])

        #creamos el dataframe de los resultados
        final = pre_final.copy()
        #tomamos los datos base y las probabilidades calculadas
        final = final[['temporada', 'round', 'local', 'visitante', 'localGoles', 'proba_local',
            'vistanteGoles', 'proba_visitante', 'proba_empate', 'ganador']]

        #le aplicamos la funcion 'prediccion' para saber quien gana
        final = prediccion(final)

        #calculamos el porcentaje de aciertos y la precision
        porcentaje = (final.prediccion == final.ganador)
        ocurrencias = list(porcentaje.value_counts())
        precision = ocurrencias[1] / sum(ocurrencias) 

        return precision, final


In [11]:
def top5(final:DataFrame)-> dict:
        '''
        funcion que en un diccionario muestra de manera descendente
        los equipos a los cuales el modelo acerto mas veces

        Entrada:
                final: Dataframe con las predicciones y los resultados reales
        
        Salida:
                top: Diccionario ordenado donde la clave es el equipo y 
                el valor es el numero de veces que el modelo acerto en ese equipo
        '''
        partidos_ganados = {}

        #tomamos todos los equipos
        equipos = set(final['local'])
        #recorremos todos los equipos
        for equipo in equipos:
                #partidos en los que juega dicho equipo 
                partidos_vis = final[final['visitante'] == equipo]
                partidos_loc = final[final['local'] == equipo]
                
                #partidos en donde el modelo acerto con el equipo
                #  como visitante y partidos en donde acerto como local
                ganados_vis = len(partidos_vis[partidos_vis['ganador'] == partidos_vis['prediccion']])
                ganados_loc = len(partidos_loc[partidos_loc['ganador'] == partidos_loc['prediccion']])
                
                #numero de partidos en los que el modelo acerto en dicho equipo
                aciertos = ganados_vis + ganados_loc

                partidos_ganados[equipo] = aciertos

        #usamos Counter para poder ordenar el diccionario
        top = Counter(partidos_ganados).most_common()
        return top 

In [12]:
def pres_recall(final:DataFrame) -> float:
    '''
    Funcion que calcula el porcentaje de aciertos sobre los locales, visitantes o empate,
    usando metricas de evaluacion, en particular la de precision y el recall

    Entrada:
            final: Dataframe con las predicciones y los resultados reales

    Salida:
            pres_loc: presicion de aciertos sobre los locales

            pres_vis: presicion de aciertos sobre los visitantes
            
            pres_emp: presicion de aciertos sobre los empates

            recall_loc: recall de aciertos sobre los locales
            
            recall_vis: recall de aciertos sobre los visitantes
            
            recall_emp: recall de aciertos sobre los empates
    '''

    #para cada papel, contamos el numero de veces que el modelo acerto
    loc_acertados = len(final[(final['ganador'] == final['prediccion']) & (final['ganador'] == 'Local')])
    vis_acertados = len(final[(final['ganador'] == final['prediccion']) & (final['ganador'] == 'Visitante')])
    emp_acertados = len(final[(final['ganador'] == final['prediccion']) & (final['ganador'] == 'Empate')])

    #contamos tambien los falsos positivos 
    loc_falsoPos = len(final[(final['ganador'] != final['prediccion']) & (final['prediccion'] == 'Local')])
    vis_falsoPos = len(final[(final['ganador'] != final['prediccion']) & (final['prediccion'] == 'Visitante')])
    emp_falsoPos = len(final[(final['ganador'] != final['prediccion']) & (final['prediccion'] == 'Empate')])

    #contamos los falsos negativos
    loc_falsoNeg = len(final[(final['ganador'] != final['prediccion']) & (final['ganador'] == 'Local')])
    vis_falsoNeg = len(final[(final['ganador'] != final['prediccion']) & (final['ganador'] == 'Visitante')])
    emp_falsoNeg = len(final[(final['ganador'] != final['prediccion']) & (final['ganador'] == 'Empate')])
    #para no dividir entre cero, consideramos los casos cuando el modelo tiene un 0% 
    # de aciertos 
    if vis_acertados + vis_falsoPos == 0 and emp_acertados + emp_falsoPos == 0:
        pres_loc = loc_acertados/ (loc_acertados + loc_falsoPos)
        pres_vis = 0
        pres_emp = 0
    
    elif vis_acertados + vis_falsoPos == 0 and emp_acertados + emp_falsoPos != 0:
        pres_loc = loc_acertados/ (loc_acertados + loc_falsoPos)
        pres_vis = 0
        pres_emp = emp_acertados/ (emp_acertados + emp_falsoPos)

    elif vis_acertados + vis_falsoPos != 0 and emp_acertados + emp_falsoPos == 0:
        pres_loc = loc_acertados/ (loc_acertados + loc_falsoPos)
        pres_vis = vis_acertados/ (vis_acertados + vis_falsoPos)
        pres_emp = 0
    
    #caso cuando ninguno de los tres porcentajes es cero
    else:
        pres_loc = loc_acertados/ (loc_acertados + loc_falsoPos)
        pres_vis = vis_acertados/ (vis_acertados + vis_falsoPos)
        pres_emp = emp_acertados/ (emp_acertados + emp_falsoPos)

    #ahora calculamos el recall
    if vis_acertados + vis_falsoPos == 0 and emp_acertados + emp_falsoNeg == 0:

        recall_loc = loc_acertados/ (loc_acertados + loc_falsoNeg)
        recall_vis = 0
        recall_emp = 0
    
    elif vis_acertados + vis_falsoPos == 0 and emp_acertados + emp_falsoNeg != 0:
        recall_loc = loc_acertados/ (loc_acertados + loc_falsoNeg)
        recall_vis = 0
        recall_emp = emp_acertados/ (emp_acertados + emp_falsoNeg)

    elif vis_acertados + vis_falsoPos != 0 and emp_acertados + emp_falsoNeg == 0:
        recall_loc = loc_acertados/ (loc_acertados + loc_falsoNeg)
        recall_vis = vis_acertados/ (vis_acertados + vis_falsoNeg)
        recall_emp = 0
    
    #caso cuando ninguno de los tres porcentajes es cero
    else:
        recall_loc = loc_acertados/ (loc_acertados + loc_falsoNeg)
        recall_vis = vis_acertados/ (vis_acertados + vis_falsoNeg)
        recall_emp = emp_acertados/ (emp_acertados + emp_falsoNeg)

    return pres_loc, pres_vis, pres_emp, recall_loc, recall_vis, recall_emp

In [13]:
def grafica(final:DataFrame, temp_predecir:str, jornada = 0):
    '''
    Funcion que grafica el porcentaje de aciertos de cada joranda

    Entrada:
            final: Dataframe con las predicciones y los resultados reales

            temp_predecir: Temporada a predecir

            jornada: Numero de jornada hasta la cual se quiere visualizar,
            si no se indica se muestran todas por defecto
    '''
    #creamos una lista con los 32 rounds y otra donde se almacenaran los porcentajes
    rounds = [i+1 for i in range(32)]
    porcentajes = []
    #si la joranda es 0, entoces mostramos las 32
    if jornada == 0:
        for round in range(1, 33):
            #calculamos el porcentaje de aciertos
            dfjornada = final[final['round'] == round]
            acertados = (dfjornada.prediccion == dfjornada.ganador)
            ocurrencias = list(acertados.value_counts())
            porc =  sum(acertados) / sum(ocurrencias)
            porcentajes.append(porc)
        #graficamos
        plt.rcdefaults()
        plt.rcParams.update({'axes.facecolor':'#D5D5D5'})
        plt.figure(figsize=(15,4))
        plt.plot(rounds, porcentajes, linewidth = 3, linestyle = '-.', color = '#840FCB' )
        plt.scatter(rounds, porcentajes, marker = '*', s = [100], color = '#35B900')
        plt.grid()
        plt.title(f'Porcentaje de aciertos del modelo en la temporada {temp_predecir}')
        plt.xlabel('Jornada')
        plt.ylabel('Porcentaje')
        plt.xticks(rounds)
        plt.show()

    #si se especifica la jornada, se muestran los resultados hasta esa jornada
    elif jornada >0:
        for round in range(1, jornada + 1):
            #calculamos el porcentaje de aciertos
            dfjornada = final[final['round'] == round]
            acertados = (dfjornada.prediccion == dfjornada.ganador)
            ocurrencias = list(acertados.value_counts())
            porc =  sum(acertados) / sum(ocurrencias)
            porcentajes.append(porc)
        #graficamos
        plt.rcdefaults()
        plt.rcParams.update({'axes.facecolor':'#D5D5D5'})
        plt.figure(figsize=(15,4))
        plt.plot(rounds[: jornada], porcentajes, linewidth = 3, linestyle = '-.', color = '#840FCB' )
        plt.scatter(rounds[: jornada], porcentajes, marker = '*', s = [400], color = '#35B900')
        plt.grid()
        plt.title(f'Porcentaje de aciertos del modelo en la temporada {temp_predecir}')
        plt.xlabel('Jornada')
        plt.ylabel('Porcentaje')
        plt.xticks(rounds[: jornada])
        plt.show()
        

In [14]:
def grafica_top5 (top):
        '''
        Funcion que grafica el top 5 de equipos a los cuales
        el modelo acerto mas

        Entrada:
                top: diccionario ordenado de mayor a menor por valor,
                en donde la clave es el nombre del equio y el valor es
                el numero de aciertos que el modelo hizo
        '''

        #creamos listas para almacenar el nombre del equipo y los aciertos
        equipos = []
        aciertos = []
        #indice que nos ayuda a recorrer el diccionario
        i = 1
        for equipo in top:
                if i <= 5:
                        equipos.append(equipo[0])
                        aciertos.append(equipo[1])
                i += 1

        #graficamos
        plt.rcdefaults()
        plt.rcParams.update({'axes.facecolor':'#D5D5D5'})
        plt.barh(equipos, aciertos, color = '#FF6927')
        plt.grid( linestyle = '--', linewidth = 0.5)
        plt.xlabel('Cantidad de predicciones acertadas')
        plt.ylabel('Nombre del equipo')
        plt.xticks([i*2 for i in range(17)])
        plt.title('Aciertos de la predicción en los equipos del Top 5')
        plt.show()

In [18]:
def modelo(temp_entr:str, temp_predecir:str, historico:DataFrame, jornada = 0):

    '''
    Funcion que muestra los resultados del modelo

    Entrada: 
            temp_entr: Temporada para entrenar al modelo

            temp_predecir: Temporada a predecir

            historico: Dataframe con todos los partidos de las temporadas

            jornada: Jornada hasta la cual se quiere visualizar
    '''
    precision, final = resultado(temp_entr, temp_predecir, historico)
    top = top5(final)
    pres_loc, pres_vis, pres_emp, a, b, c= pres_recall(final)

    print(f'***RESULTADOS TEMPORADA {temp_predecir}\n')
    print(f'****Porcenaje de aciertos: {precision*100}%')
    print('***Top 5 equipos con más aciertos:\n')
    
    #imprimimos solo los primeros 5 resultados del diccionario top
    i = 1
    for equipo in top:
        if i <= 5:
            print(f'Para el equipo {equipo[0]} el modelo acertó a {equipo[1]} partidos de 32')
            i += 1
    
    print('')
    print(f'***El porcentaje de aciertos sobre los locales es {round(pres_loc*100, 2)}%')
    print(f'***El porcentaje de aciertos sobre los visitantes es {round(pres_vis*100, 2)}%')
    print(f'***El porcentaje de aciertos sde empate es {round(pres_emp*100, 2)}%')

    grafica(final, temp_predecir, jornada)
    grafica_top5(top)

    if jornada != 0:
        print(f'***Predicciones para la jornada {jornada + 1}')
        jor_predecir = final[final['round'] == jornada + 1]
        print(jor_predecir[['temporada', 'round', 'local', 'visitante', 'prediccion']])

In [16]:
def barcelona(historico: DataFrame):
    '''
    Funcion que grafica el numero de aciertos por temporada
    para el equipo Barcelona. Toma desde la temporada 2000-01
    hasta la 2016-17

    Entrada: 
            historico: Dataframe con todos los partidos de las temporadas
    '''

    #creamos una lista con las temporadas desde el 2000
    temporadas = ['200'+str(i) + '-0'+ str(i+1) for i in range(9) ]
    temporadas.append('2009-10')
    temporadas1 = ['20'+str(i) + '-'+ str(i+1) for i in range(10, 17)]
    temporadas.extend(temporadas1)

    #lista donde se va almacenar el numero de aciertos del modelo
    aciertos_lista = []
    x = [i for i in range(len(temporadas))]
    for i in range(len(temporadas)-1):
        #usando la funcion 'resultado' tomamos el dataframe con predicciones 
        precision, final = resultado(temporadas[i], temporadas[i+1], historico)

        #tomamos todos los partidos en los que ha participados el Barcelona en la temporada dada
        vis = final[final['visitante'] == 'Barcelona']
        local = final[final['local'] == 'Barcelona']

        #tomamos los partidos a los cuales el modelo acerto
        ganados_vis = len(vis[vis['ganador'] == vis['prediccion']])
        ganados_loc = len(local[local['ganador'] == local['prediccion']])

        #los contamos y añadimos a la lista
        aciertos = ganados_vis + ganados_loc
        aciertos_lista.append(aciertos)

    #graficamos los resultados
    plt.rcdefaults()
    plt.rcParams.update({'axes.facecolor':'#D5D5D5'})
    plt.plot(x[1:], aciertos_lista, color = '#2737B0', linewidth = 2, linestyle = '-.')
    plt.bar(x[1:], aciertos_lista, color = '#03AAB8')
    plt.grid( linestyle = '--', linewidth = 0.5)
    plt.title('Aciertos del modelo para el equipo Barcelona')
    plt.ylabel('Cantidad de predicciones acertadas')
    plt.xlabel('Temporada')
    plt.xticks(x, temporadas,  rotation=45)
    plt.yticks([j*2 for j in range(14)])
    plt.show()

In [17]:
def promedios_fin(historico: DataFrame):
    '''
    Funcion que imprime el promedio de la precision y el recall
    del modelo de las ultimas 5 temporadas para los locales,
    visitas y empates

    Entrada:
            historico: Dataframe con todos los partidos de las temporadas
    '''

    precision_total_vis = 0
    precision_total_loc = 0
    precision_total_emp = 0

    recall_total_vis = 0
    recall_total_loc = 0
    recall_total_emp = 0

    #lista que contiene las 5 temporadas finales
    temporadas = ['20'+str(i) + '-'+ str(i+1) for i in range(12, 18)]
    for i in range(len(temporadas)-1):
        #usando la funcion 'resultado' tomamos el dataframe con predicciones 
        precision, final = resultado(temporadas[i], temporadas[i+1], historico) 
        #obtenemos los valores de la precision y recall con la funcion pres_recall 
        pres_loc, pres_vis, pres_emp, recall_loc, recall_vis, recall_emp = pres_recall(final)

        #vamos acumulando los resultados
        precision_total_vis += pres_vis
        precision_total_loc += pres_loc
        precision_total_emp += pres_emp

        recall_total_vis += recall_vis
        recall_total_loc += recall_loc
        recall_total_emp += recall_emp

    #sacamos el porcentajes e imprimimos
    precision_total_vis = precision_total_vis / 5
    precision_total_loc = precision_total_loc / 5
    precision_total_emp = precision_total_emp / 5

    recall_total_vis = recall_total_vis / 5
    recall_total_loc = recall_total_loc / 5
    recall_total_emp = recall_total_emp / 5

    print('***El porcentaje de la precisión del modelo en de la temporada 2012-13 a la 2016-17 son:')
    print(f'* {precision_total_loc*100} para los locales')
    print(f'* {precision_total_vis*100} para las visitas')
    print(f'* {precision_total_emp*100} para los empates')

    print()
    print('***El recall_total_emp del recall del modelo en de la temporada 2012-13 a la 2016-17 es de:')
    print(f'* {recall_total_loc*100} para los locales')
    print(f'* {recall_total_vis*100} para las visitas')
    print(f'* {recall_total_emp*100} para los empates')